# Red Wine Quality Classifier
## Introduction:
Red wine is a popular type of wine made from dark-colored grapes, known for its rich flavor and complexity. Wine quality is typically evaluated based on factors such as aroma, taste, color, and acidity,etc.
We use this dataset to answer “how can we use this dataset so that a business can predict the quality of wine they produce based on its chemical composition”.

## There are 12 variables in this dataset:
    Input variables (based on physicochemical tests):
            1 - fixed acidity ：refers to the non-volatile acids present in wine, which do not evaporate easily.
            2 - volatile acidity：refers to the amount of acetic acid in wine, which can lead to an unpleasant vinegar taste if present in excessive amounts.
            3 - citric acid ：is found in small quantities in wine and can add a sense of freshness and flavor to it.
            4 - residual sugar： refers to the amount of sugar left over after the fermentation process is complete，wines with more than 45 grams per liter are considered sweet.
            5 - chlorides : refer to the amount of salt present in wine.
            6 - free sulfur dioxide : helps prevent microbial growth and wine oxidation.
            7 - total sulfur dioxide :refers to the amount of both free and bound forms of SO2. Concentrations over 50 ppm can be detected in the taste and aroma of wine.
            8 - density: Density of wine is close to that of water and depends on its alcohol and sugar content.
            9 - pH :measures the acidity or basicity of wine on a scale from 0 to 14
            10 - sulphates: are an additive in wine that can contribute to sulfur dioxide gas levels, acting as an antimicrobial and antioxidant.
            11 - alcohol:  percentage of alcohol content is an important factor to consider when assessing wine quality.
    Output variable (based on sensory rating): 
            12 - quality :(score between 0 and 10) 

## Preliminary exploratory data analysis: 
(Demonstrate that the dataset can be read from the web into R. Clean and wrangle your data into a tidy format. Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.)

## Methods: 
First split the data set into training and data set. Use training set to preprocess and create model specification. Next use cross-validation on training set to determine the best K nearest-neighbor value to use when creating a classifier, which ensures the highest prediction accuracy. Lastly use the tuned classifier with K value that gives the highest prediction accuracy on the test setto predict the quality of the newly introduced wine. 
<br>
The columns we will use when classifying will be pH, alcohol, sulphates, residual sugar, volatile acidity, and quality. The classifier variable is quality while the predictor variables are pH, alcohol, sulphates, residual sugar, volatile yacidity, and quality.
Since the rating of the quality is a scale from 0 to 10 (distinct numerical values) and is being compared to different wines numbered 1 to n, where n is the total number of wines used (ordered, distinct numerical values), the best visualization to visualize the results would be a line plot. The visualization of the results would include the different wines (numbered) on the x-axis and their ratings from 0-10 on the y-axis. It will show the relative relationship between the different wines, with the newly introduced wine included, and their relative wine quality ratings.
<br>
## Expected outcomes and significance: 
In this research, we expected to find which physicochemical properties make a high-quality red wine, and what chemical compositions are required for a high-quality red wine. 
Once we know what physicochemical properties contribute to a good quality score, businesses will be able to use this information to help them produce higher quality wine.
Based on our findings, we came up with two questions that can be worked on in the future: 
1. “What role does each chemical composition play in the formation of a high-quality red wine?” 
2. “What relationships between these properties contribute to the formation of a high-quality wine?”

## *Reading and Tidying Data*

In [8]:
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)

wine_quality <- read_csv2("data/winequality-red.csv")
colnames(wine_quality) <- make.names(colnames(wine_quality))
wine_quality

wine_quality <- select(wine_quality, volatile.acidity, pH, alcohol, sulphates, residual.sugar, quality) |>
                         mutate(across(1:6, as.numeric))
wine_quality

ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 1599 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (5): volatile acidity, citric acid, chlorides, density, sulphates
dbl (2): total sulfur dioxide, quality

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
74,0.7,0,19,0.076,11,34,0.9978,351,0.56,94,5
78,0.88,0,26,0.098,25,67,0.9968,32,0.68,98,5
78,0.76,0.04,23,0.092,15,54,0.997,326,0.65,98,5
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
63,0.51,0.13,23,0.076,29,40,0.99574,342,0.75,11,6
59,0.645,0.12,2,0.075,32,44,0.99547,357,0.71,102,5
6,0.31,0.47,36,0.067,18,42,0.99549,339,0.66,11,6


volatile.acidity,pH,alcohol,sulphates,residual.sugar,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.70,351,94,0.56,19,5
0.88,32,98,0.68,26,5
0.76,326,98,0.65,23,5
⋮,⋮,⋮,⋮,⋮,⋮
0.510,342,11,0.75,23,6
0.645,357,102,0.71,2,5
0.310,339,11,0.66,36,6


## **Wrangling and Visualizing**
Summary table including mean column values, grouped by quality.
Plot of number of observations for each quality of wine.

quality,mean_pH,mean_acidity,mean_alcohol,mean_sulphates,mean_sugar
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,277.7000,0.8845000,1.711000e+02,0.5700000,64.60000
4,285.5094,0.6939623,9.866038e+01,0.5964151,24.90566
5,297.4816,0.5770411,9.308957e+01,0.6209692,27.25110
6,298.5721,0.4974843,3.434692e+12,0.6753292,22.47962
7,294.7035,0.4039196,6.817420e+11,0.7412563,37.33668
8,293.2222,0.4233333,9.094444e+01,0.7677778,23.77778


quality,observations
<dbl>,<int>
3,10
4,53
5,681
6,638
7,199
8,18


In [9]:
wine_summary <- group_by(wine_quality, quality) |>
                summarize(mean_pH = mean(pH), 
                          mean_acidity = mean(volatile.acidity), 
                          mean_alcohol = mean(alcohol),
                         mean_sulphates = mean(sulphates),
                         mean_sugar = mean(residual.sugar))
wine_summary

wine_grouped <- group_by(wine_quality, quality) |>
                summarize(observations = n())
wine_grouped

quality_plot <- ggplot(wine_grouped, aes(x = quality, y = observations)) +
                        geom_bar(stat = "identity") +
                        xlab("Quality of Red Wine") +
                        ylab("Number of Observations") + 
                        theme(text = element_text(size=20))
quality_plot